# OLLAMA - REST API Approach

> This page was generated from [ollama-interactive-inference/ollama-sif-api-ibex.ipynb](https://github.com/kaust-rccl/Data-science-onboarding/tree/main/notebooks/inference/ollama-interactive-inference/ollama-sif-api-ibex.ipynb). You can [view or Download notebook](https://github.com/kaust-rccl/Data-science-onboarding/tree/main/notebooks/inference/ollama-interactive-inference/ollama-sif-api-ibex.ipynb). Or [view it on nbviewer](https://nbviewer.org/github/kaust-rccl/Data-science-onboarding/tree/main/notebooks/inference/ollama-interactive-inference/ollama-sif-api-ibex.ipynb)

## Objective
In this notebook, we are going to use Ollama using REST API approach 

## Initial Setup
If you haven't installed conda yet, please follow [`How to Setup Conda on Ibex Guide`](https://docs.hpc.kaust.edu.sa/soft_env/prog_env/python_package_management/conda/ibex.html) to get started.

After conda has been installed, save the following environment yaml file on Ibex under the name *ollama_env.yaml*

```yml
name: ollama_env
channels:
  - conda-forge
dependencies:
  - _libgcc_mutex=0.1
  - _openmp_mutex=4.5
  - _python_abi3_support=1.0
  - anyio=4.11.0
  - argon2-cffi=25.1.0
  - argon2-cffi-bindings=25.1.0
  - arrow=1.4.0
  - asttokens=3.0.0
  - async-lru=2.0.5
  - attrs=25.4.0
  - babel=2.17.0
  - beautifulsoup4=4.14.2
  - bleach=6.2.0
  - bleach-with-css=6.2.0
  - brotli-python=1.1.0
  - bzip2=1.0.8
  - ca-certificates=2025.10.5
  - cached-property=1.5.2
  - cached_property=1.5.2
  - certifi=2025.10.5
  - cffi=2.0.0
  - charset-normalizer=3.4.4
  - comm=0.2.3
  - cpython=3.14.0
  - debugpy=1.8.17
  - decorator=5.2.1
  - defusedxml=0.7.1
  - exceptiongroup=1.3.0
  - executing=2.2.1
  - fqdn=1.5.1
  - h11=0.16.0
  - h2=4.3.0
  - hpack=4.1.0
  - httpcore=1.0.9
  - httpx=0.28.1
  - hyperframe=6.1.0
  - idna=3.11
  - importlib-metadata=8.7.0
  - ipykernel=7.0.1
  - ipython=9.6.0
  - ipython_pygments_lexers=1.1.1
  - isoduration=20.11.0
  - jedi=0.19.2
  - jinja2=3.1.6
  - json5=0.12.1
  - jsonpointer=3.0.0
  - jsonschema=4.25.1
  - jsonschema-specifications=2025.9.1
  - jsonschema-with-format-nongpl=4.25.1
  - jupyter-lsp=2.3.0
  - jupyter_client=8.6.3
  - jupyter_core=5.9.1
  - jupyter_events=0.12.0
  - jupyter_server=2.17.0
  - jupyter_server_terminals=0.5.3
  - jupyterlab=4.4.9
  - jupyterlab_pygments=0.3.0
  - jupyterlab_server=2.27.3
  - keyutils=1.6.3
  - krb5=1.21.3
  - lark=1.3.0
  - ld_impl_linux-64=2.44
  - libedit=3.1.20250104
  - libexpat=2.7.1
  - libffi=3.4.6
  - libgcc=15.2.0
  - libgcc-ng=15.2.0
  - libgomp=15.2.0
  - liblzma=5.8.1
  - libmpdec=4.0.0
  - libsodium=1.0.20
  - libsqlite=3.50.4
  - libstdcxx=15.2.0
  - libstdcxx-ng=15.2.0
  - libuuid=2.41.2
  - libzlib=1.3.1
  - markupsafe=3.0.3
  - matplotlib-inline=0.1.7
  - mistune=3.1.4
  - nbclient=0.10.2
  - nbconvert-core=7.16.6
  - nbformat=5.10.4
  - ncurses=6.5
  - nest-asyncio=1.6.0
  - notebook-shim=0.2.4
  - openssl=3.5.4
  - overrides=7.7.0
  - packaging=25.0
  - pandocfilters=1.5.0
  - parso=0.8.5
  - pexpect=4.9.0
  - pickleshare=0.7.5
  - pip=25.2
  - platformdirs=4.5.0
  - prometheus_client=0.23.1
  - prompt-toolkit=3.0.52
  - psutil=7.1.0
  - ptyprocess=0.7.0
  - pure_eval=0.2.3
  - pycparser=2.22
  - pygments=2.19.2
  - pysocks=1.7.1
  - python=3.14.0
  - python-dateutil=2.9.0.post0
  - python-fastjsonschema=2.21.2
  - python-gil=3.14.0
  - python-json-logger=2.0.7
  - python-tzdata=2025.2
  - python_abi=3.14
  - pytz=2025.2
  - pyyaml=6.0.3
  - pyzmq=27.1.0
  - readline=8.2
  - referencing=0.37.0
  - requests=2.32.5
  - rfc3339-validator=0.1.4
  - rfc3986-validator=0.1.1
  - rfc3987-syntax=1.1.0
  - rpds-py=0.27.1
  - send2trash=1.8.3
  - setuptools=80.9.0
  - six=1.17.0
  - sniffio=1.3.1
  - soupsieve=2.8
  - stack_data=0.6.3
  - terminado=0.18.1
  - tinycss2=1.4.0
  - tk=8.6.13
  - tomli=2.3.0
  - tornado=6.5.2
  - traitlets=5.14.3
  - typing-extensions=4.15.0
  - typing_extensions=4.15.0
  - typing_utils=0.1.0
  - tzdata=2025b
  - uri-template=1.3.0
  - urllib3=2.5.0
  - wcwidth=0.2.14
  - webcolors=24.11.1
  - webencodings=0.5.1
  - websocket-client=1.9.0
  - yaml=0.2.5
  - zeromq=4.3.5
  - zipp=3.23.0
  - zstandard=0.25.0
  - zstd=1.5.7
  - pip:
      - annotated-types==0.7.0
      - ollama==0.6.0
      - pydantic==2.12.3
      - pydantic-core==2.41.4
      - typing-inspection==0.4.2
```

Run the following command to build the conda environment:
```bash
conda env create -f ollama_env.yaml
```

## Starting JupyterLab
Follow [`Guide: Using Jupyter on Ibex`](https://docs.hpc.kaust.edu.sa/soft_env/job_schd/slurm/interactive_jobs/jupyter.html#job-on-ibex) to start JupyterLab on a an Ibex GPU node using your conda environment instead of *'machine_learning'* module.

By making the following changes to the Jupyter launch script:
```bash
#module load machine_learning/2024.01
conda activate ollama_env
```

## Starting The Ollama Server
Start the OLLAMA REST API server using the following bash script in a terminal:
```bash
#!/bin/bash

# Cleanup process while exiting the server
cleanup() {
    echo "🧹   Cleaning up before exit..."
    # Put your exit commands here, e.g.:
    rm -f $OLLAMA_PORT_TXT_FILE
    # Remove the Singularity instance
    singularity instance stop $SINGULARITY_INSTANCE_NAME
}
trap cleanup SIGINT  # Catch Ctrl+C (SIGINT) and run cleanup

# User Editable Section
# 1. Make target directory on /ibex/user/$USER/ollama_models_scratch to store your Ollama models
export OLLAMA_MODELS_SCRATCH=/ibex/user/$USER/ollama_models_scratch
mkdir -p $OLLAMA_MODELS_SCRATCH
# End of User Editable Section

SINGULARITY_INSTANCE_NAME="ollama"
OLLAMA_PORT_TXT_FILE='ollama_port.txt'

# 2. Load Singularity module
module load singularity

# 3. Pull OLLAMA docker image
singularity pull docker://ollama/ollama

# 4. Change the default port for OLLAMA_HOST: (default 127.0.0.1:11434)
export PORT=$(python -c 'import socket; s=socket.socket(); s.bind(("", 0)); print(s.getsockname()[1]); s.close()')

# 5. Copy the assigned port, it will be required in the second part during working on the notebook.
echo "$PORT" > $OLLAMA_PORT_TXT_FILE

echo "OLLAMA PORT: $PORT  -- Stored in $OLLAMA_PORT_TXT_FILE"

# 6. Define the OLLAMA Host
export SINGULARITYENV_OLLAMA_HOST=127.0.0.1:$PORT

# 7. Change the default model directory stored: (default ~/.ollama/models/manifests/registry.ollama.ai/library)
export SINGULARITYENV_OLLAMA_MODELS=$OLLAMA_MODELS_SCRATCH

# 8. Create an Instance:
singularity instance start --nv -B "/ibex/user:/ibex/user" ollama_latest.sif $SINGULARITY_INSTANCE_NAME

# 7. Run the OLLAMA REST API server on the background
singularity exec instance://$SINGULARITY_INSTANCE_NAME bash -c "ollama serve"
```

> Note: Save the above script in a file called start_ollama_server.sh

```bash
# Run the script to start the Ollama server.
bash start_ollama_server.sh
```

The script has the following:
- A user editable section, where the user defines Ollama models scratch directory.
- The allocated port is saved in a temporary ollama_port.txt file, in order to be used in the Python notebook to read the assigned port to Ollama server.
- Cleanup section in order to stop the singularity instance when the script is terminated with CTRL+C.

## Using REST API Requests
Follow the following Python notebook below, it contains the codes for:
- Initialization Setup.
- List local models.
- Pull models.
- Testing connection to the Ollama server.
- Chat with the models.

### 1. Initialization
1. Define the base URL for the remote Ollama Server.
2. Testing the Ollama server connectivity.

In [7]:
# 1.1- Define the base URL for the remote Ollama Server
with open("ollama_port.txt") as f :
    PORT = f.read().strip()
    
BASE_URL=f"http://127.0.0.1:{PORT}"
print(BASE_URL)

http://127.0.0.1:51023


In [8]:
# 1.2- Testing the Ollama server connectivity
import requests

try:
    r = requests.get(BASE_URL)
    print("Ollama is running!", r.status_code)
except requests.ConnectionError as e:
    print("Ollama is NOT reachable:", e)

Ollama is running! 200


### 2. Get a List of Local Models
- Get a list of locally available Ollama models.
- Locally available models are located under path: */ibex/user/$USER/ollama_models_scratch*
- To change the location for pulled models, modify the variable *OLLAMA_MODELS_SCRATCH* in the script*start_ollama_server.sh*

In [9]:
# Get a list of downloaded models
def get_local_models(base_url: str = BASE_URL):
    """
    Returns a list of locally available Ollama Models.

    Returns:
        list: A list of model names as strings

    Raises:
        RuntimeError: If there is a failure to connect the Ollama server.
    """
    r = requests.get(f"{base_url}/api/tags")
    if r.ok:
        models = r.json().get("models", [])
        return [m["name"] for m in models]
    else:
        raise RuntimeError(f"Failed to list models: {r.text}")

get_local_models()

['gemma3:270m', 'qwen3:0.6b']

### 3. Pull The Model
- Pull a model from the Ollama server and stream the download progress.
- Please refer to [Ollama Library](https://ollama.com/library) to check available models.

In [10]:
# Pull the required model
import requests

def pull_model(model: str, base_url: str =BASE_URL) -> list:
    """
    Pull a model from the Ollama server and stream the download progress.

    Args:
        model_name (str): Name of the model to pull.
        base_url (str, optional): Base URL of the Ollama server. Defaults to BASE_URL.

    Returns:
        list: A list of strings representing the streamed output lines.

    Raises:
        requests.HTTPError: If the server response indicates failure.
    """
    url = f"{base_url}/api/pull"
    response = requests.post(url, json={"name": model}, stream=True)

    if response.status_code != 200:
        raise requests.HTTPError(f"Failed to pull model '{model}': {response.text}")

    output_lines = []
    for line in response.iter_lines():
        if line:
            decoded = line.decode("utf-8")
            print(decoded)
            output_lines.append(decoded)

    return output_lines

In [ ]:
# Usage
model = "phi3:3.8b"
output_logs = pull_model(model=model)

### 4. Running a Sample Query
- Send a single chat prompt to a specified Ollama model and stream the response.

In [5]:
import requests
import json
from typing import List

def chat_once(model: str, prompt: str, base_url: str = BASE_URL) -> List[str]:
    """
    Send a single chat prompt to a specified Ollama model and stream the response.

    Args:
        model (str): Name of the Ollama model to use.
        prompt (str): User input to send to the model.
        base_url (str, optional): Base URL of the Ollama server. Defaults to BASE_URL.

    Returns:
        List[str]: List of streamed output chunks from the model.

    Raises:
        requests.HTTPError: If the server response status is not 200.
    """
    url = f"{base_url}/api/chat"
    response = requests.post(
        url,
        json={"model": model, "messages": [{"role": "user", "content": prompt}]},
        stream=True
    )

    if response.status_code != 200:
        raise requests.HTTPError(f"Failed to chat with model '{model}': {response.text}")

    output_lines = []
    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode('utf-8'))
            if "message" in data:
                content = data["message"]["content"]
                print(content, end="", flush=True)  # Stream to console
                output_lines.append(content)

    print()  # Newline after full response
    return output_lines

In [10]:
# Usage
model="qwen3:0.6b"
prompt= "How old are you"
output_logs = chat_once(model=model, prompt=prompt)

<think>
Okay, the user is asking how old I am. I need to respond in a friendly and helpful way. First, I should acknowledge their question. Then, I can mention that I'm an AI assistant and that I'm only a few years old. I should add something about my capabilities, like being available 24/7, but also mention that I can assist with various tasks. It's important to keep the tone positive and offer support. Let me make sure I'm concise and friendly.
</think>

I'm a very small AI assistant, but I'm only a few years old! I'm available 24/7 and can help with a wide range of tasks. Let me know how I can assist you! 😊


### 5. Interactive Chat with Ollama Models
- This function enables a live, interactive conversation with a local Ollama LLM model.
- Users can type messages in the terminal, and the model streams its responses in real time.
- Features:
    - Maintains conversation history between user and model.
    - Supports multiple local models (must be pulled beforehand).
    - Type 'exit' or 'quit' to end the session.
    - Returns the full conversation history for further processing or logging.

In [7]:
import requests
import json
from typing import List, Dict

def ollama_chat(
    model: str,
    base_url: str = BASE_URL,
    system_prompt: str = 'You are a helpful assistant. You only give a short sentence by answer.'
) -> List [Dict[str, str]]:
    """
    Start an interactive chat session with a local Ollama model via HTTP streaming.

    This function streams responses from the model in real time, maintains conversation
    history, and allows the user to exit by typing 'exit'. A system prompt can guide
    the assistant's behavior.

    Args:
        model (str): Name of the local Ollama model to use.
        base_url (str, optional): Base URL of the Ollama server. Defaults to BASE_URL.
        system_prompt (str, optional): Instruction for the assistant. Defaults to a short-answer style.

    Returns:
        List[Dict[str, str]]: Full conversation history as a list of messages with roles ('user' or 'assistant').

    Raises:
        ValueError: If the requested model is not in the local models list.
        requests.HTTPError: If the chat request fails.
    """
    # Validate model existence
    if model not in get_local_models():
        raise ValueError(f"Requested model '{model}' is not in the local list. Pull the model first!")

    # Initialize message history
    history: List[Dict[str, str]] = []

    print("🤖 Chat started — type 'exit' to quit.\n")
    
    while True:
        user_input = input("👤 You: ").strip()
        if user_input.lower() == 'exit':
            print("👋 Goodbye!")
            break
    
        # Compose full message payload with system + history
        request_messages = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': user_input}]
    
        # Start request
        try:
            response = requests.post(
                f"{base_url}/api/chat",
                json={"model": model, "messages": request_messages},
                stream=True
            )

            if response.status_code != 200:
                raise requests.HTTPError(f"Chat request failed: {response.text}")

            assistant_reply = ""
            print("🤖 Ollama:", end=" ", flush=True)
    
            for line in response.iter_lines():
                if line:
                    data = json.loads(line.decode("utf-8"))
                    if "message" in data and "content" in data["message"]:
                        chunk = data["message"]["content"]
                        assistant_reply += chunk
                        print(chunk, end='', flush=True)
    
            print("\n")
    
            # Add interaction to message history
            history.append({'role': 'user', 'content': user_input})
            history.append({'role': 'assistant', 'content': assistant_reply})
    
        except Exception as e:
            print("\n⚠️ Error:", e)

    return history


In [8]:
# Usage
model = "qwen3:0.6b"
history = ollama_chat(model='qwen3:0.6b')

🤖 Chat started — type 'exit' to quit.



👤 You:  Hello, What is HPC?


🤖 Ollama: <think>
Okay, the user is asking what HPC is. I need to explain High-Performance Computing. Let me start by defining it clearly. High-Performance Computing refers to the use of powerful computers to solve complex problems quickly. I should mention the different types, like supercomputers, clusters, and cloud computing. Also, I should highlight its benefits like speed, efficiency, and impact on various fields. Make sure to keep it simple and concise. Let me check if I need to mention any specific applications or technologies. No, just the general benefits. Alright, time to put this into a short sentence.
</think>

High-Performance Computing (HPC) is a technology that uses powerful computers to solve complex problems faster and more efficiently.



👤 You:  exit


👋 Goodbye!
